# Creation
Algorithm: Given an operator $\rho_{i,j}\left|i\right>\left<j\right|$ with indices $i,j\in\mathbb Z_d^n$:
(<mark>todo: update max bond dimension to shrink after middle index reached</mark>)
1. Reshape: $\rho_{i,j}= P_{(i_1\cdots i_n),(j_1\cdots j_n)}$ has dimensions $d^n\times d^n$
2. Permute: $P_{(i_1\cdots i_n),(j_1\cdots j_n)}= T_{(i_1j_1),\cdots ,(i_nj_n)}$ has dimensions $d^2\times\cdots \times d^2$ ($n$ times)
3. Collect: $T_{(i_1j_1),(i_2j_2\cdots i_nj_n)}$ has dimensions $d^2\times d^{2(n-1)}$
4. First site:
	- SVD: $T_{(i_1j_1),(i_2j_2\cdots i_nj_n)}=\sum_{\alpha\in \mathbb Z_{D_1}} U_{(i_1j_1),\alpha}S_{\alpha,\alpha} (V^\dagger)_{\alpha,(i_2j_2\cdots i_nj_n)}$, where $D_1\in \{1,\cdots,d^2\}$ is determined by truncation 
	- Save: $M^{(1)}_{\alpha,u,\beta}=\delta_{\alpha,1}U_{u,\beta}$ has dimensions $1\times d^2\times D_1$
	- Update: $T^{(1)}_{(\alpha i_2j_2),(i_3j_3\cdots i_nj_n)}=S_{\alpha,\alpha}(V^\dagger)_{\alpha,(i_2j_2\cdots i_nj_n)}$ has dimensions $D_1 d^2\times d^{2(n-2)}$
5. Inner sites: For $k\in \{2,\cdots,n-1\}$:
	- SVD: $T^{(k)}_{(\alpha i_kj_k),(i_{k+1}j_{k+1}\cdots i_nj_n)}=\sum_{\beta\in \mathbb Z_{D_k}} U_{(\alpha i_kj_k),\beta}S_{\beta,\beta} (V^\dagger)_{\beta,(i_{k+1}j_{k+1}\cdots i_nj_n)}$, where $D_k\in\{1,\cdots,D_{k-1}d^2\}$
	- Save: $M^{(k)}_{\alpha,u,\beta}=U_{(\alpha u),\beta}$ has dimensions $D_{k-1}\times d^2\times D_k$
	- Update: $T_{(\alpha i_{k+1}j_{k+1}),(i_{k+2}j_{k+2}\cdots i_nj_n)}=S_{\alpha,\alpha} (V^\dagger)_{\alpha,(i_{k+1}j_{k+1}\cdots i_nj_n)}$ has dimensions $D_k d^2\times d^{2(n-k-1)}$
6. Last site:
	- Save: $M^{(n)}_{\alpha,u,\beta}=T^{(n-1)}_{(\alpha,u),\beta}$ has dimensions $D_{n-1}\times d^2\times 1$ (note that $T^{(n-1)}_{(\alpha i_nj_n),\beta}$ has dimensions $D_{n-1} d^2\times 1$)
7. Return $\{M^{(k)}\}_{k\in\mathbb Z_n}$

In [1]:
import numpy as np
from numpy import linalg as la

def state_to_local_tensor(state, n, d):
    tensor_shape = tuple([d]*(2*n))
    tensor = state.reshape(tensor_shape)
    return tensor_to_local_tensor(tensor, n, d)
    
def tensor_to_local_tensor(tensor, n, d):
    local_axes = [int(i/2) if i%2 == 0 else int(n + (i/2)) for i in range(2*n)]
    original_axes = [i for i in range(2*n)]
    return np.moveaxis(tensor, original_axes, local_axes)

# Given a density matrix, return its matrix product operator representation
# Indices are stored in the following format: (bond,phys,bond)
# Usage: state_to_mpo(state, n, d, verbose) or state_to_mpo(state, n, d, verbose, max_bd). verbose defaults to False if not given.
def state_to_mpo(state, n, d, verbose=False, **kwargs):
    # If not given a maximum bond dimension, set it to the maximum possible - d^(4n) (TODO)?
    #TODO: add a truncation magnitude cuttoff instead of just max_bd
    max_bd = kwargs.get('max_bd', d**(4*n))
    
    # Local tensor representation of state                                                                    
    T = state_to_local_tensor(state, n, d)
    
    # Preparing for first SVD: Constructing environment tensor for first site
    d2 = d**2
    T = T.reshape((d2,d2**(n-1)))
    if verbose:
        print(0, "Initial T:\t\t\t\t", T.shape)

    # First site
    mpo = [0]*n
    U, S, Vt = la.svd(T, full_matrices = False)
    U = U[:,:max_bd]
    S = S[:max_bd]
    Vt = Vt[:max_bd,:]
    mpo[0] = U.reshape((1,U.shape[0],U.shape[1]))
    T = np.dot(np.diag(S),Vt).reshape(S.size*d2,int(Vt.shape[1]/d2))
    if verbose:
        print(1, U.shape, Vt.shape, "\t->", mpo[0].shape, "\t", T.shape, "\t", S.size)

    # Interior sites
    for i in range(1,n-1):
            U, S, Vt = la.svd(T, full_matrices = False)
            U = U[:,:max_bd]
            S = S[:max_bd]
            Vt = Vt[:max_bd,:]
            mpo[i] = U.reshape((int(U.shape[0]/d2),d2,U.shape[1]))
            T = np.dot(np.diag(S),Vt).reshape((S.size*d2,int(Vt.shape[1]/d2)))
            if verbose:
                print(i+1, U.shape, Vt.shape, "\t->", mpo[i].shape, "\t", T.shape, "\t", S.size)

    # Last site
    mpo[n-1] = np.dot(np.diag(S),Vt).reshape((S.size, Vt.shape[0], 1))
    if verbose:
        print(n, "Final Matrix:\t\t  ", mpo[n-1].shape)

    return mpo

We can run this algorithm on a randomly generated density matrix:

In [2]:
# Generate a mixed state of randomly sampled pure qudit states                                                 
# d - onsite hilbert space dimension
# N - number of sites
# M - number of pure states used
def random_mixed_state(N,d=3,M=10):
        states = [random_state(N,d) for i in range(M)]
        weights = [np.random.uniform(low=0.0,high=1.0) for i in range(M)]
        total_weight = sum(weights)
        state = np.zeros((d**N,d**N))
        for i in range(M):
                state = state + (weights[i]/total_weight) * np.outer(states[i],states[i].conj())
        return state

# Generate a random qudit state
# d - onsite hilbert space dimension
# N - number of sites
def random_state(N,d=3):
        normals = [np.random.standard_normal(size=d**N) for i in range(2)]
        state = np.array([normals[0][i] + 1j * normals[1][i] for i in range(d**N)])
        state /= la.norm(state)
        return state

I should expect to see the following dimensions of my matrices and local tensor after performing each SVD for $d=3,n=5$: The density matrix has dimensions $3^5\times 3^5$ and the environment tensor $T$ starts out with dimensions $9\times 9^4$. After each step, we should expect $D_k=9^k$ <mark>(TODO: update this)</mark>

In [3]:
d = 3
n = 2
rho = random_mixed_state(n,d)
m_rho = state_to_mpo(rho,n,d,verbose=True)

0 Initial T:				 (9, 9)
1 (9, 9) (9, 9) 	-> (1, 9, 9) 	 (81, 1) 	 9
2 Final Matrix:		   (9, 9, 1)


To check that the MPO procedure returned an operator corresponding to the same operator which we input into the algorithm, we can use numpy's unravel_index method to compute each matrix element and check:

In [4]:
def check_coefficients(mpo, state):
    # Calculate tensor of MPO coefficients
    mpo_tensor_shape = tuple([d**2]*n)
    mpo_tensor = np.zeros(mpo_tensor_shape, dtype=np.complex128)
    for u in range(d**(2*n)):
        # Calculate coefficient
        index = np.unravel_index(u,mpo_tensor_shape)
        coefficient = np.identity(1,dtype=np.complex128)
        for i in range(n):
            coefficient = np.dot(coefficient, mpo[i][:,index[i],:])
        mpo_tensor[index] = coefficient[0][0]
        
    # Reshape into state
    state_tensor_shape = tuple([d]*(2*n))
    mpo_state = mpo_tensor.reshape(state_tensor_shape)
    tensor_axes = [i for i in range(2*n)]
    T_axes = [int(i/2) if i%2 == 0 else int(n + (i/2)) for i in range(2*n)]
    mpo_state = np.moveaxis(mpo_state, T_axes, tensor_axes)
    mpo_state = mpo_state.reshape((d**n,d**n))
    print(np.allclose(mpo_state, state))
    
check_coefficients(m_rho, rho)

True


# Inner products
The Frobenius norm overlap between two operators is defined to be
$$\left<\rho|\sigma\right>=\mathrm{Tr}\left[\rho^\dagger\sigma\right]=\sum_u \left(\prod_i \rho^{(i)u_i}\right)^*\left(\prod_j \sigma^{(i)u_j}\right),$$
where $*$ denotes element-wise complex conjugation. This can be calculated in a linear number of computations in $d$ and $n$ and cubic in $D^\sigma$ and $D^\rho$ like so:
1. $M_{\alpha\beta} = \sum_{u\in \mathbb Z_d}(\rho^{(1)}_{1u\alpha})^*\sigma^{(1)}_{1u\beta}$
2. For $k\in\{2,\cdots,n\}:$
    - $M_{\alpha\beta}\leftarrow \sum_{u\in \mathbb Z_d,\mu\in\mathbb Z_{D^\rho_k},\mu\in\mathbb Z_{D^\sigma_k}}(\rho^{(k)}_{\mu u \alpha})^*M_{\mu \nu}\sigma^{(k)}_{\nu u \beta}$
3. Return $M_{11}$

Note that the matrix elements are stored like so: $\rho^{(k)u}_{\alpha\beta}=\rho^{(k)}_{\alpha u \beta}$, and $D^\rho_k$ denotes the bond dimension of $\rho^{(k)}$ at site $k$ 

In [5]:
# Frobenius inner product tr[A^\dagger B] between two states A and B represented by mpos 
# norm: Frobenius normalization of local basis elements used in MPO representation (norm = Tr[A_u^\dagger A_v])
# Assumptions:
#       len(m1) = len(m2)                                       [same length]
#       m1[i].shape = m2[i].shape = (bond,phys,bond)            [same bond and phys dim at each site]
#       m1[0].shape = (1,_,_) and m1[-1].shape = (_,_,1)        [closed boundary conditions]
def inner_prod(m1, m2, norm=1):
        # Extract length, phys dim, and first site bond dim
        n = len(m1)

        #print(m1[0][0].shape, m2[0][0].shape)
        # First site
        M = np.tensordot(m1[0][0].conj(),m2[0][0],axes=([0,0]))      # sum over physical indices (grab the first and only matrix due to fixed bdry cond.)

        #print(0,m1[0].shape,M.shape)
        # Rest of contraction
        for i in range(1,n):
                M = np.tensordot(M,m1[i].conj(),axes=([0,0]))
                M = np.tensordot(M,m2[i],axes=([0,1],[0,1]))
                #print(i,m1[i].shape, M.shape)
        return M[0][0]*norm
print(np.absolute(inner_prod(m_rho,m_rho) - np.trace(np.dot(rho.conj().T,rho))) < 1e-14)

True


## Moving to phase space

We formed our MPO like so: $\rho_{ij}\left|i\right>\left<j\right|=\rho_{(i_1\cdots i_n)(j_1\cdots j_n)}(\left|i_1\right>\otimes \cdots \otimes \left|i_n\right>)(\left<j_1\right|\otimes \cdots \otimes \left<j_n\right|)=\sum_{i_kj_k\alpha _k}\rho^{(1)}_{\alpha_0(i_1j_1)\alpha_1}\left|i_1\right>\left<j_1\right|\otimes\cdots\otimes \rho^{(n)}_{\alpha_{n-1}(i_nj_n)\alpha_n}\left|i_n\right>\left<j_n\right|$. This gave us a matrix product operator of the form $\sum_{u,\alpha} \otimes_k \rho^{(k)u_k}_{\alpha_k\alpha_{k+1}}C_{u_k}$, where $C_u=C_{du_1+u_2}=\left|u_1\right>\left<u_2\right|$. We want to represent it in phase space, i.e. we want to find the MPO $\tilde{\rho}$ satisfying $\sum_{u,\alpha} \otimes_k \rho^{(k)u_k}_{\alpha_k\alpha_{k+1}} C_{u_k}=\sum_{u,\alpha} \otimes_k \tilde{\rho}^{(k)u_k}_{\alpha_k\alpha_{k+1}} A_{u_k}$. Taking the Frobenius inner product with $A_v$ on both sides of this equation shows that the following choice works:

$$\tilde{\rho}^{(k)u}_{\alpha\beta}=d^{-1}\sum_v\rho^{(k)v}_{\alpha\beta}\mathrm{Tr}\left[C_uA_v\right]=d^{-1}\sum_v\rho^{(k)v}_{\alpha\beta}(A_v)_{u_1,u_2}$$

The last equality arises because $\mathrm{Tr}[C_u A_v]=\sum_k \left<k|u_1\right>\left<u_2|A_v |k\right>=\left<u_1|A_v|u_2\right>=(A_v)_{u_1,u_2}$, so the change-of-basis coefficients are proportional to the matrix elements of the phase space operators. 

Since $\mathrm{Tr}[A_uA_v]=d\delta_{u,v}$, the phase space basis is not normalized. This means that we pick up a Hilbert space factor when computing inner products:

$$\mathrm{Tr}[\rho^\dagger \sigma]=d^n\left<\rho|\sigma\right>_A,$$

where $\left<\rho|\sigma\right>_A$ denotes the contraction of the matrix product representations of the states $\rho$ and $\sigma$ in the phase space basis $A_u$.

In [6]:
import cmath
import math

d = 3   # qutrit

# qutrit X and Z matrices
omega = cmath.exp(cmath.pi*2j/d)
X = np.array([[0,0,1],[1,0,0],[0,1,0]])
Z = np.array([[1,0,0],[0,omega,0],[0,0,omega**2]])

# 1-qudit pauli matrices
T = np.array([[omega**(-2*a*b)*np.dot(la.matrix_power(Z,a),la.matrix_power(X,b)) for b in range(d)] for a in range(d)])

# 1-qudit phase space operators (TODO: change the sum below to a numpy sum)
A_0 = (1./d)*sum([sum([T[a,b] for b in range(d)]) for a in range(d)])
A = np.zeros((d**2,d,d), dtype=np.complex128)
for u in range(d**2):
    index = np.unravel_index(u,(d,d))
    A[u] = np.dot(T[index],np.dot(A_0,np.conj(T[index].T)))
ps_coefficients = 1./d*A.reshape((d**2,d**2))

# Computational basis C_u = |u_1><u_2|
#comp_basis = np.zeros((d,d,d,d), dtype=np.complex128)
#for i in range(d):
#    for j in range(d):
#        comp_basis[i,j,i,j] = 1
#comp_basis = comp_basis.reshape((d**2,d,d))

# Change of basis coefficients Tr[A_u^\dagger C_u] = Tr[A_u C_u]
#ps_coefficients = np.zeros((d**2,d**2),dtype=np.complex128)
#for i in range(d**2):
#    for j in range(d**2):
#        ps_coefficients[i,j] = 1./d*np.trace(np.dot(A[i],comp_basis[j]))

# change of basis coefficients
# TODO: change this to a numpy array
#ps_coefficients = [[np.trace(np.dot(A[u],comp_basis[v])) for u in range(d**2)] for v in range(d**2)]

#TODO: get rid of for loops
def basis_change(mpo, coefficients):
    n = len(mpo)
    _,d,_ = mpo[0].shape
    
    res = [None]*n
    for k in range(n):
        res[k] = np.zeros(mpo[k].shape, dtype=np.complex128)
        temp = np.tensordot(mpo[k],coefficients,axes=([1,1])) # sum over physical indices
        temp = np.moveaxis(temp, [0,1,2], [0,2,1])
        res[k] = temp
    return res

Some checks to make sure that we did the change of basis correctly:

In [7]:
# Check properties of phase space basis: Hermitian, Tr[A_uA_v] = d*delta_uv, Tr[A_u] = 1
for i in range(d**2):
    if np.absolute(np.trace(A[i]) - 1.) > 1e-14:
        print(i, "Trace error: expected 1, got " + repr(np.trace(A[i])))
    for j in range(d**2):
        if abs(np.trace(np.dot(A[i],A[j])).imag) > 1e-14:
            print(i,j,"complex trace error!")
        if not(np.allclose(A[i].conj().T,A[i])):
            print(i,j,"hermiticity error!")
        if i == j:
            if abs(np.trace(np.dot(A[i],A[j])).real - d) > 1e-14:
                print(i,j,"normalization error!")
        else:
            if abs(np.trace(np.dot(A[i],A[j])).real) > 1e-14:
                print(i,j,"normalization error!")

# Check that inner product in new basis agrees with old basis
m_rho_ps = basis_change(m_rho, ps_coefficients)
if (abs(inner_prod(m_rho_ps,m_rho_ps,norm=d**n).imag) > 1e-12):
    print("Inner product in new basis is imaginary")
if (abs(inner_prod(m_rho_ps,m_rho_ps,norm=d**n).real - inner_prod(m_rho,m_rho).real) > 1e-12):
    print("Inner product in new basis does not match")

We can now calculate the Wigner function and estimations of magic:

In [8]:
# n-dimensional phase space operators:
A_n = np.zeros((d**(2*n),d**n,d**n),dtype=np.complex128)
tensor_shape = tuple([d**2]*n)
for u in range(d**(2*n)):
    index = np.unravel_index(u,tensor_shape)
    temp = np.identity(1,dtype=np.complex128)
    for i in range(n):
        temp = np.kron(temp, A[index[i]])
    A_n[u] = temp
    
# Check properties of n-qudit phase space basis: Hermitian, Tr[A_uA_v] = d^n*delta_uv, Tr[A_u] = 1
for i in range(d**(2*n)):
    if np.absolute(np.trace(A_n[i]) - 1.) > 1e-14:
        print(i, "Trace error: expected 1, got " + repr(np.trace(A_n[i])))
    for j in range(d**(2*n)):
        if abs(np.trace(np.dot(A_n[i],A_n[j])).imag) > 1e-14:
            print(i,j,"complex trace error!")
        if not(np.allclose(A_n[i].conj().T,A_n[i])):
            print(i,j,"hermiticity error!")
        if i == j:
            if abs(np.trace(np.dot(A_n[i],A_n[j])).real - (d**n)) > 1e-14:
                print(i,j,"normalization error!")
        else:
            if abs(np.trace(np.dot(A_n[i],A_n[j])).real) > 1e-14:
                print(i,j,"normalization error!")
    
# Wigner function, magic estimates
def wigner(state, n, d):
    W = np.zeros((d**(2*n)), dtype=np.complex128)
    M = 0. #old magic quantifier ("mana" in arxiv:1307.7171)
    N = 0. #new magic quantifier
    for u in range(d**(2*n)):
        W[u] = 1./(d**n)*np.trace(np.dot(state,A_n[u]))
        if W[u].real < 0:
            M -= W[u].real
            N += (W[u].real)**2
    N = math.sqrt(N)
    return W, M, N

We now generate a random hermitian operator (not necessarily normalized) $\sigma$ whose matrix product operator representation in the phase space basis has only non-negative elements. We then vary $\sigma$ to reduce the squared distance

$$d(\rho,\sigma)^2=\mathrm{Tr}[(\rho-\sigma)^\dagger(\rho-\sigma)]=\mathrm{Tr}[\rho^\dagger\rho]-\mathrm{Tr}[\rho^\dagger\sigma]-\mathrm{Tr}[\sigma^\dagger\rho]+\mathrm{Tr}[\sigma^\dagger\sigma],$$

in the space of non-negative matrix product operators (i.e. constrained optimization with the constraints $\sigma^{(k)}_{\alpha,u,\beta}\geq 0$). Note that $\sigma$ is a hermitian operator because it has a real Wigner function: 
$$\sigma^{(k)}_{\alpha,u,\beta}\geq 0\Rightarrow\left(\sum_\alpha \prod_k \sigma^{(k)}_{\alpha_k,u_k,\alpha_{k+1}}\right)\geq 0\ \forall  u\Rightarrow \mathrm{Tr}[\sigma A_{u}]\in\mathbb R\ \forall u$$
Since the phase space operators are hermitian and complete, this implies that
$$\sigma^\dagger=\left(d^{-n}\sum_u \mathrm{Tr}[\sigma A_u]A_u\right)^\dagger=d^{-n}\sum_u \mathrm{Tr}[\sigma A_u]^*A_u^\dagger=d^{-n}\sum_u \mathrm{Tr}[\sigma A_u]A_u=\sigma.$$
Using this and the fact that trace is cyclic, the squared distance simplifies to

$$d(\rho,\sigma)^2=\mathrm{Tr}[\rho^\dagger\rho]-2\mathrm{Tr}[\sigma^\dagger\rho]+\mathrm{Tr}[\sigma^\dagger\sigma]$$

In [9]:
import scipy as sp
from scipy.optimize import minimize

#TODO: seed 

def dist_sq(A, B, norm):
    return inner_prod(A,A,norm) - 2*inner_prod(A,B,norm) + inner_prod(B,B,norm)

def dist(A, B, norm):
    return math.pow(dist_sq(A,B,norm),1./2)

def cost_fct(i, x, A, B, norm):
    A[i] = x.reshape(A[i].shape)
    return inner_prod(A, B, norm)

# Generate random MPO with fixed bond dimension and random positive coefficients
def gen_mpo(n, d, bd):
    mpo = [None] * n
    mpo[0] = np.random.rand(1,d,bd)
    for i in range(1,n-1):
        mpo[i] = np.random.rand(bd,d,bd)
    mpo[n-1] = np.random.rand(bd,d,1)
    return mpo
    
def optimize(A, B, nswp = 20, opt_method = 'L-BFGS-B'):
    # The true distance between B and the positive-repr states is N:
    _, _, true_dist = wigner(B)
    print("Initial distance: %g, True distance: %g" % (dist(A,B), true_dist))

    # Optimize D(A||B) on A to find closest positive repr MPO to B
    print("Optimization method: %s" % opt_method)
    print("Sweep Site Distance Dist-True_Dist")
    #print("Half-sweep D(A||B) D(A||B)-D(A*||B) E(A) H_2(A)\n")
    sys.stdout.flush()

    n = len(A)
    _,d,_ = A.shape
    dn = d**n
    b = [(0, None) for i1 in range(D_A*(d**2))] #todo: need to impose constraints dependent on bond dim at that site
    start_time = time.time()
    for i in range(nswp):
        for j in range(n):
            A[j] = minimize(lambda x:cost_fct(0,x,A,B,dn), A[j], method=opt_method, bounds=b).x
            print("%d %d %g %g %g %g" % (i, j, dist(A,B,dn), dist(A,B,dn)-true_dist)) #vn_entropy(A), r_entropy(A,2)))
            sys.stdout.flush()

    print("\nTime elapsed during optimization: %g" % (time.time() - start_time))

# Debugging
Unfortunately, the squared distance comes out to be complex, which isn't good.. let's try to debug:

In [10]:
mA = gen_mpo(n,d**2,10)
norm = d**n
mB = m_rho_ps
print(inner_prod(mA,mA,norm))
print(inner_prod(mB,mB,norm))
print(inner_prod(mA,mB,norm))
print("Magnitude of complex value: " + repr(np.amax( [np.amax(np.abs(np.imag(mB[j]) ) ) for j in range(n) ] )))

# Check that an mpo in the phase space basis represents a hermitian operator
def check_real_coefficients(mpo):
    good = True
    tensor_shape = tuple([d**2]*n)
    for u in range(d**(2*n)):
        index = np.unravel_index(u,tensor_shape)
        coefficient = np.identity(1,dtype=np.complex128)
        for i in range(n):
            coefficient = np.dot(coefficient, mpo[i][:,index[i],:])
        if (coefficient[0][0].imag > 1e-16):
            good = False
            print("Coefficient " + repr(index) + " is complex with value " + repr(coefficient[0][0]))
    if good:
        print("Coefficents were all real!")
        
check_real_coefficients(mA)
check_real_coefficients(mB)


5772.1612845543095
(0.22591152150753221+6.933347799794049e-33j)
(25.969902878311586+1.5217918517284053e-14j)
Magnitude of complex value: 2.5303758779070102e-14
Coefficents were all real!
Coefficents were all real!


Is my random state even hermitian? What's going on here..

In [11]:
print(np.allclose(rho,rho.T.conj()))

True


Ok... what about the Wigner function? Does it agree with these coefficients?

In [12]:
# Check that an mpo in the phase space basis represents a hermitian operator
def check_against_wigner(mpo, state):
    W, _, _ = wigner(state, n, d)
    good = True
    tensor_shape = tuple([d**2]*n)
    for u in range(d**(2*n)):
        index = np.unravel_index(u,tensor_shape)
        coefficient = np.identity(1,dtype=np.complex128)
        for i in range(n):
            coefficient = np.dot(coefficient, mpo[i][:,index[i],:])
        if (np.absolute(W[u] - coefficient[0][0]) > 1e-16):
            good = False
            print(repr(index) + ": " + repr(coefficient[0][0]) +"\t" + repr(W[u]) + "\t" + ("%.2E %.2E" % (np.absolute(W[u] - coefficient[0][0]),W[u]/coefficient[0][0].real)))
    if good:
        print("Coefficents all agreed!")
        
check_against_wigner(mB, rho)

(0, 3): (0.01563895617438192+5.593487220855884e-18j)	(0.01222883711574474+6.1679056923619804e-18j)	3.41E-03 7.82E-01
(0, 4): (0.02416569498320674+6.479794209343468e-18j)	(0.009624775349714532-7.709882115452476e-18j)	1.45E-02 3.98E-01
(0, 5): (-0.008087045924695791+3.1853047481638414e-18j)	(0.00590376391270925+1.3877787807814457e-17j)	1.40E-02 -7.30E-01
(0, 6): (0.012228837115744752-2.158573627875372e-18j)	(0.015638956174381925+1.1275702593849246e-17j)	3.41E-03 1.28E+00
(0, 7): (0.009624775349714523-1.0506942526496597e-17j)	(0.024165694983206733+1.1564823173178713e-17j)	1.45E-02 2.51E+00
(0, 8): (0.005903763912709267-9.767927514558938e-19j)	(-0.008087045924695772+0j)	1.40E-02 -1.37E+00
(1, 3): (0.0368733843789048+2.4913765669650785e-17j)	(0.0022601069703421135+1.734723475976807e-18j)	3.46E-02 6.13E-02
(1, 4): (-0.0024262568961690195-1.0858783808435481e-17j)	(0.03943098549439924+2.929755203871941e-17j)	4.19E-02 -1.63E+01
(1, 5): (0.015255805596471614+2.28458325302051e-17j)	(0.01784421407

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  del sys.path[0]


# Direct MPO representation of W(u)
Let's just make an MPO out of the Wigner function representation:

In [17]:
def state_to_wigner_mpo(state, n, d):
    W, _, _ = wigner(state, n, d)
    #print(W.shape)
    #state_shape = (d**2, d**2)
    #W_state = W.reshape(state_shape)
    return state_to_mpo(W, n, d)
    
state_to_wigner_mpo(rho, n, d)

(81,)


[array([[[-0.37018595-2.78259926e-16j, -0.4218597 -5.90064963e-16j,
           0.13496913+3.49124393e-16j,  0.06640584+7.96264918e-16j,
          -0.68406267+3.73902784e-16j,  0.02009222+2.87262527e-16j,
          -0.03966835-1.17538920e-15j, -0.3237487 +1.74826976e-16j,
          -0.29603734-4.98781927e-16j],
         [-0.32916503-2.21772007e-16j, -0.42050428-8.03078871e-16j,
           0.31111866+7.02197885e-16j, -0.02141422+4.12225678e-17j,
           0.03429754-1.07044077e-16j, -0.28734005-3.86801232e-17j,
           0.08766727+2.60923163e-15j,  0.64481661+3.60773277e-16j,
           0.33220251+5.57295811e-16j],
         [-0.3834534 -1.90195933e-16j,  0.45186693+5.13063644e-16j,
          -0.0418593 -4.31538678e-16j,  0.34186753+6.00759930e-17j,
          -0.26816095-5.29991597e-16j,  0.33709489-4.83914119e-16j,
          -0.32886518-8.66136968e-15j,  0.09386097-6.19913285e-16j,
           0.47712547+1.47819855e-15j],
         [-0.38055098-1.48518305e-16j, -0.35360774-4.47868237e-1

In [18]:
wm_rho = state_to_wigner_mpo(rho, n, d)
print(np.absolute(inner_prod(mB,mB,norm) - inner_prod(m_rho,m_rho)) < 1e-14)
print(np.absolute(inner_prod(wm_rho,wm_rho,norm) - inner_prod(m_rho, m_rho)) < 1e-14)
print(np.absolute(inner_prod(mB,mB,norm)))
print(np.absolute(inner_prod(wm_rho,wm_rho,norm)))
print(np.allclose(wm_rho[0],mB[0]))
print((wm_rho[0] - mB[0]).real)

(81,)
True
True
0.22591152150753221
0.2259115215075319
False
[[[-0.25391837 -0.30404666  0.11951558  0.44667419 -0.54430356
    0.15843215 -0.29312948 -0.27796715 -0.059218  ]
  [ 0.00912701 -0.56688694  0.24875167 -0.21223224 -0.02894069
   -0.11646061 -0.21044621  0.69127936  0.15072665]
  [-0.33387478  0.77971976 -0.20338736  0.27322112  0.083835
    0.26299956 -0.31561537  0.13951506  0.23027927]
  [-0.20675979 -0.64218479 -0.25320084  0.52091375  0.67564287
    0.16795463 -0.06046877 -0.14543442 -0.32127092]
  [ 0.00646019 -0.06673039 -0.54975981  0.26442478 -0.05643696
   -0.63522405  0.36824686  0.145655    0.37463934]
  [-0.01545148  0.27074779  0.35829707 -0.38711846  0.13202206
    0.31547608  0.82049333 -0.57908231  0.2982909 ]
  [-0.23395478  0.04965867  0.09035426 -0.53156509  0.36579772
   -0.21778107  0.08646989 -0.1682811   0.28077822]
  [-0.20688469  0.54093891 -0.18933798 -0.00575264 -0.36052494
    0.02306689  0.1127026   0.42678566 -0.84473927]
  [-0.07888502  0.562

In [19]:
check_against_wigner(wm_rho, rho)


(0, 3): (0.01827351377661725+1.2335811384723995e-17j)	(0.01222883711574474+6.1679056923619804e-18j)	6.04E-03 6.69E-01
(0, 4): (0.03744154271616791+1.8503717077085947e-17j)	(0.009624775349714532-7.709882115452476e-18j)	2.78E-02 2.57E-01
(0, 5): (0.03431384528037674+1.2721305490496583e-17j)	(0.00590376391270925+1.3877787807814457e-17j)	2.84E-02 1.72E-01
(0, 6): (0.0063702805923005965+1.9274705288631923e-18j)	(0.015638956174381925+1.1275702593849246e-17j)	9.27E-03 2.45E+00
(0, 7): (-0.011792342739148486+1.927470528863092e-18j)	(0.024165694983206733+1.1564823173178713e-17j)	3.60E-02 -2.05E+00
(0, 8): (0.0015474197028598389+8.688221687967081e-18j)	(-0.008087045924695772+0j)	9.63E-03 -5.23E+00
(1, 0): (0.012228837115744733+6.1679056923619704e-18j)	(0.018273513776617258+1.2335811384723961e-17j)	6.04E-03 1.49E+00
(1, 1): (0.00962477534971456-7.709882115452456e-18j)	(0.03744154271616794+1.850371707708594e-17j)	2.78E-02 3.89E+00
(1, 2): (0.005903763912709266+1.3877787807814418e-17j)	(0.034313845

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  del sys.path[0]


In [20]:
dist_sq(wm_rho, mA, norm)

(5723.318149369891+2.7434806851305426e-14j)

In [21]:
dist_sq(mB, mA, norm)

(5720.447390319194+3.0435837034568105e-14j)